The manual: http://data.atoc.org/sites/all/themes/atoc/files/RSPS5045.pdf

The data: http://data.atoc.org/fares-data

In [1]:
import pandas as pd
import sys
sys.path.append("./..")

from src.data_prep import create_records_table
import config.indices_of_records as dicts

### Flow file

In [2]:
out = pd.read_table("../data/RJFAF575/RJFAF575.FFL")

In [3]:
flow_data = out.copy()

In [4]:
flow_data.rename(columns={"/!! Start of file                                                               ":"records"}, inplace=True)

In [5]:
lengths_of_records = flow_data["records"].str.len()

In [6]:
flow_records = create_records_table(
    relevant_dictionary=dicts.dict_flow_records,
    table_with_records=flow_data, 
    lengths_of_records=lengths_of_records)

fare_records = create_records_table(
    relevant_dictionary=dicts.dict_fare_records,
    table_with_records=flow_data, 
    lengths_of_records=lengths_of_records)

In [7]:
merged_data = flow_records.merge(
    fare_records,
    on=["flow_id", "update_marker"])

In [8]:
merged_data.head()

,records_x,update_marker,record_type_x,origin_code,destination_code,route_code,status_code,usage_code,direction,end_date,...,toc,cross_london_ind,non_standard_disc_ind,publication_ind,flow_id,records_y,record_type_y,ticket_code,fare,restriction_code
0,RF0027003201000000AS3112299902012020ATO01Y0000002,R,F,0027,0032,01000,000,A,S,31122999,...,ATO,0,1,Y,0000002,RT00000027TF00005420,T,7TF,00005420,
1,RF0027003201000000AS3112299902012020ATO01Y0000002,R,F,0027,0032,01000,000,A,S,31122999,...,ATO,0,1,Y,0000002,RT00000027TS00003610,T,7TS,00003610,
2,RF0027003301000000AS3112299902012020ATO01Y0000034,R,F,0027,0033,01000,000,A,S,31122999,...,ATO,0,1,Y,0000034,RT00000347TF00006360,T,7TF,00006360,
3,RF0027003301000000AS3112299902012020ATO01Y0000034,R,F,0027,0033,01000,000,A,S,31122999,...,ATO,0,1,Y,0000034,RT00000347TS00004240,T,7TS,00004240,
4,RF0027003401000000AS3112299902012020ATO01Y0000121,R,F,0027,0034,01000,000,A,S,31122999,...,ATO,0,1,Y,0000121,RT00001217TF00007790,T,7TF,00007790,


## Locations file
Locations, including group locations

In [9]:
locations_file = pd.read_table("../data/RJFAF575/RJFAF575.LOC")

In [10]:
locations_file.rename(
    columns={'/!! Start of file                                                               ':"records"},
    inplace=True)

In [11]:
lengths_of_records_locations = locations_file["records"].str.len()

In [12]:
locations_records = create_records_table(
    relevant_dictionary=dicts.dictionary_locations,
    table_with_records=locations_file, 
    lengths_of_records=lengths_of_records_locations)

In [13]:
associated_stations_records = create_records_table(
    relevant_dictionary=dicts.dict_associated_stations,
    table_with_records=locations_file, 
    lengths_of_records=lengths_of_records_locations)

In [14]:
synonym_records = create_records_table(
    relevant_dictionary=dicts.dict_synonym,
    table_with_records=locations_file, 
    lengths_of_records=lengths_of_records_locations)

### Routes

In [15]:
routes_file = pd.read_table("../data/RJFAF575/RJFAF575.RTE")

In [16]:
routes_file.rename(
    columns={'/!! Start of file                                                               ':"records"},
    inplace=True)

In [17]:
lengths_of_records_routes = routes_file["records"].str.len()

In [18]:
routes_records = create_records_table(
    relevant_dictionary=dicts.dict_routes,
    table_with_records=routes_file, 
    lengths_of_records=lengths_of_records_routes)

route_excl_records = create_records_table(
    relevant_dictionary=dicts.dict_route_exclusions,
    table_with_records=routes_file, 
    lengths_of_records=lengths_of_records_routes)